## Introduction

In the EDA and ModelTraining notebooks, we explored a dataset comprising hypothetical customers of the InterConnect phone and internet service. This document constitutes a report on the plan and its execution, decisions made regarding challenges in the process, and final models.



## The Plan and its Execution

### EDA
The standard beginning to any data project is the loading and cleaning of the data. There were four dataframes as part of the synthetic InterConnect dataset, alignable based on the `customerID` column identifying each unique customer. Two of the dataframes outlined user internet and phone plan usage, which were only populated if customers had the respective plans. Some users had one and not the other, so upon merge to a master dataframe, many users had NaN values for columns from these dataframes. We transformed all NaN values to False values for those columns and added boolean columns (`HasInternet` and `HasPhone`) to designate users for whom we had to transform values. This allowed us to merge the dataframes without losing meaningful information.

When we were done checking for duplicates and cleaning the final dataframe, we engineered a few meaningful features. We wanted to capture information from the `BeginDate` and `EndDate` columns that would be useful to the business interest. Since the interest is in creating a model for production predicting users who might churn their service, we shouldn't have features that won't apply to future users (only current and past users will have beginning and ending dates from 2017 to 2020). We therefore created features representing how long a user had been a customer (`DaysCustomer`) and whether they churned at a similar or dissimilar year as they signed up (`ChurnSeasonality`).

After final cleanup, the final part of the EDA was seeking to understand feature distributions through a few multivariate pairplots. This enabled us to see trends and relationships between features both as a final check and to hopefully inform model training.



### Model Training
We had some more processing to perform before we could pass data into our models. After removing a few unnecessary features only necessary for EDA, we needed to encode categorical features and scale the numeric data. 

The initial plan was to use neural network and random forest models due to the overwhelmingly boolean majority of our feature set. However, the random forest and decision tree models proved insufficient for achieving an AUC-ROC of .85 or higher, and for reasons we'll review below, we couldn't train a neural network. We had to pivot to the gradient boosting model LightGBM, which leverages categorical features remaining categorical and not numeric. Therefore we had to split the feature set into two during prep, one to pass to standard models and one to pass to LightGBM. Luckily, LightGBM did the trick and we didn't have to explore other gradient boost or gradient descent tactics.

Finally, we trained our models, tuning hyperparameters maximizing z-score, and scored them using AUC-ROC.



## Challenges

The primary challenge of this task was in choosing the best model for the data and for the constraints of the development environment. Due to an absence of GPU and memory limitations, we wanted to leverage a CPU based Keras model in the tensorflow library. However the targeted tensorflow version had compatibility issues in the anaconda package manager with numpy and other package dependencies. After some troubleshooting it became clear that a gradient boost model should be the next model to try before attempting a neural network architecture. LightGBM struggled until we expanded the hyperparameter set. However, this introduced memory issues with so many iterations passed through our grid search pipeline, and crashed the development environment. We found a balance and accomplished the desired AUC-ROC.

Future training on a similar set should use a GPU-based neural network and an environment with more memory.



## Model Performance

Our random forest and decision tree models performed adequately, but their precision was low, and their AUC-ROC scores were below 0.80. In fact, for all models trained, precision was lower than recall, meaning many false positives were identified, but the models captured true positives well. The LightGBM model performed best among the three, with a precision of 0.76, a recall of 0.85, and an AUC-ROC of 0.88 (rounded up).